<a href="https://colab.research.google.com/github/vardhanreddy2003/GPT-2Training/blob/main/LayerNormandGeLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
import torch.nn as nn

In [18]:
class LayerNorm(nn.Module):
  def __init__(self,emb_dim):
    super().__init__()
    self.eps=1e-5
    self.scale=nn.Parameter(torch.ones(emb_dim))
    self.shift=nn.Parameter(torch.zeros(emb_dim))

  def forward(self,x):
    mean=x.mean(dim=-1,keepdim=True)
    variance=x.variance(dim=-1,keepdim=True)
    norm_x=(x-mean)/torch.sqrt(variance+self.eps)
    return self.scale*norm_x+self.shift

In [26]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [27]:
class ResidualNetwork(nn.Module):

  def __init__(self,layer_sizes,use_shortcut):
    super().__init__()
    self.use_shortcut=use_shortcut
    self.layers = nn.ModuleList([
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
        ])

  def forward(self,x):

    for layer in self.layers:

      layer_output=layer(x)
      if self.use_shortcut and x.shape==layer_output.shape:
        x=x+layer_output
      else:
        x=layer_output
    return x




In [30]:
layer_sizes=[3,3,3,3,3,1]
inputs=torch.tensor([[10,2.3,3.1]])
rr=ResidualNetwork(layer_sizes,True)
rr(inputs)

tensor([[-0.]], grad_fn=<MulBackward0>)